In [1]:
!pip install transformers qwen_vl_utils accelerate>=0.26.0 PEFT -U

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00


In [3]:
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import torch
import os
from qwen_vl_utils import process_vision_info



model_name = "NAMAA-Space/Qari-OCR-0.1-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
                model_name,
                torch_dtype="auto",
                device_map="auto"
            )
processor = AutoProcessor.from_pretrained(model_name)
max_tokens = 2000

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.61k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [4]:
from transformers.image_utils import load_image

path = "/content/images/sample nyu book_page-0017.jpg"
# image = Image.open(path)
image = load_image(path)
src="image.png"

prompt = "Below is the image of one page of a document, as well as some raw textual content that was previously extracted for it. Just return the plain text representation of this document as if you were reading it naturally. Do not hallucinate."
image.save(src)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": f"file://{src}"},
            {"type": "text", "text": prompt},
        ],
    }
]
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")
generated_ids = model.generate(**inputs, max_new_tokens=max_tokens)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]
os.remove(src)
print(output_text)


فيها البلل . وهـذه النسخة في بعض المـكـاتب الموقوـفة ببغداد إذا رآـها من لا يعرف خبرها ظن أنها غريقة وليس بها سوى عرق الخطيب » منزلته
وقد بلغ فيـا طلبه من العلم منـزلة رفيعه ، قال عنه ابن خلـسان :
« كان له معرفة تامة بالأدب من النحو واللغة وغيرها »
وقال ياقوت :
« انتـهت إليه الرياسة في اللغة والأدب ، وسار ذكره في الآفاق ، ورحل الناس إليه »
وقد ولي تدريس الأدب بالنظامية ، وخزـانة السكتب بها إلى أن أدرـكته الوفاة .
أـفو الهم فيه
قال ابن خلـسان :
« وذـكره الحافظ أبو سعيد السمعاني في كتاب الذيل ، وكـتاب الأنساب ، وعدد فضـائله ثم قال :
سمعت أبا منصور محمد بن عبد الملك بن الحسن بن خيرون المقريء يـقول : أبو زـكريا يـحيي بن عليّ التبريزي ما كان بمرضي الطريقة ، وذـكر عنه أشياء ، ثم قال وذا كرت أنا مع أبي الفضل محمد بن ناصر الحافظ بما ذـكره ابن خيرون ، فـسكـت عنه ، وكأنه ما أنسـكر ماقال ، ثم قال :
ولكن كان ثـفة في اللغة وما كان يـثقله »
— م ٢ — 
